# Notebook designed to be run in colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
# Change directory to the specified path
os.chdir('/content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/saiCommandExecution')
# Verify the current working directory
print("Current working directory:", os.getcwd())
!pip install -r requirements.txt
import sys
sys.path.append('/content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/saiCommandExecution')
!pip install sacremoses
!pip install sounddevice scipy openai-whisper

import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from torch.autograd import Variable
import pickle
# Ensure Python can find your modules by adding directories to the path
sys.path.append('/content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/saiCommandExecution')
sys.path.append('/content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/saiCommandExecution/src')
sys.path.append('/content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/saiCommandExecution/src/models')
sys.path.append('/content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/saiCommandExecution/src/services')
sys.path.append('/content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/saiCommandExecution/src/utils')
%ls

Mounted at /content/drive
Current working directory: /content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/saiCommandExecution
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 15.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manyli

In [2]:
import importlib
# Import your modules
import models.cnet
import services.cnet_inference
import utils.dataloader

# Reload the modules to ensure the latest changes are picked up
importlib.reload(models.cnet)
importlib.reload(services.cnet_inference)
importlib.reload(utils.dataloader)

# Ensure that the BertLayer class is available
from models.cnet import BertLayer, Encoder, Middle, Decoder, PositionalEncoding
from src.utils.dataloader import tokenize_sample
from src.services.cnet_inference import predict_intent_and_tags, run_inference

# Now use the reloaded modules
from models import cnet
from services import cnet_inference
from utils import dataloader


In [3]:
!apt-get install -y ffmpeg
!pip install pydub scipy openai-whisper

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)


# main test

In [6]:
# import sounddevice as sd
import numpy as np
from scipy.io.wavfile import write
import whisper
import IPython.display as ipd
import torch
from pydub import AudioSegment
from pydub.playback import play
from google.colab import output
import base64
import time
global closest_command
def record_audio(duration=5, sample_rate=16000):
    # Record audio using Google Colab's built-in recorder
    print("Recording audio...")
    record_js = """
    const sleep = time => new Promise(resolve => setTimeout(resolve, time))
    const b2text = blob => new Promise(resolve => {
        const reader = new FileReader()
        reader.onload = () => resolve(reader.result)
        reader.readAsDataURL(blob)
    })

    var record = time => new Promise(resolve => {
        navigator.mediaDevices.getUserMedia({ audio: true }).then(stream => {
            recorder = new MediaRecorder(stream)
            chunks = []
            recorder.ondataavailable = e => chunks.push(e.data)
            recorder.start()
            sleep(time).then(() => {
                recorder.onstop = async () => {
                    blob = new Blob(chunks)
                    text = await b2text(blob)
                    resolve(text)
                }
                recorder.stop()
            })
        })
    })

    """
    display(ipd.Javascript(record_js))
    audio_data = output.eval_js('record(%d)' % (duration * 1000))
    audio_file_path = "/content/temp_audio.wav"
    with open(audio_file_path, "wb") as f:
        f.write(base64.b64decode(audio_data.split(",")[1]))
    return audio_file_path

def main_text_only():

    try:
        text_input = "start a one hour timer and name it shabadoo"
        closest_command = run_inference(text_input, model)
        print("Interpreted command:", closest_command)
        return closest_command
    except KeyboardInterrupt:
        print("Stopping...")

def main():

    try:
        print("Listening...")
        duration = 5  # Adjust duration as needed

        # Record audio from the microphone
        audio_file_path = record_audio(duration=duration)

        # Play the recorded audio
        display(ipd.Audio(audio_file_path))

        # Process the audio using Whisper
        print("Processing audio with Whisper...")
        start_time = time.time()
        result = whisper_model.transcribe(audio_file_path)
        end_time = time.time()
        transcription_time = end_time - start_time
        user_speech = result["text"]
        print("Heard:", user_speech)
        print(f"Transcription took {transcription_time:.2f} seconds")
        if user_speech:
            closest_command = run_inference(user_speech, model)
            print("Interpreted command:", closest_command)
            return closest_command
    except KeyboardInterrupt:
        print("Stopping...")

# Run the main function
model_path = "/content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/models/ctranfinal_bert_in_model_v1"
bert_addr = '/content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/bert_models/bert-large-uncased'
model = cnet.CNet(model_path=model_path, bert_addr=bert_addr)
model = model.cuda()

# Load the Whisper model
# whisper_model = whisper.load_model("large")

# Move the Whisper model to CUDA
# if torch.cuda.is_available():
    # whisper_model = whisper_model.cuda()

In [7]:
cls_cmd =main_text_only()


example input:BOS start a one hour timer and name it shabadoo EOS	 O O O O O O O O O O open_app


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:306: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1031.)
  return F.conv1d(input, weight, bias, self.stride,
/content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/saiCommandExecution/src/models/cnet.py:164: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:78.)
  newtensor = torch.cuda.FloatTensor(batch_size, length,ENV_HIDDEN_SIZE).fill_(0.) # size of newtensor same as original


Interpreted command: {'intent': 'set_timer', 'tags': {'B-timer_length': ['one'], 'I-timer_length': ['hour'], 'B-timer_name': ['shabadoo']}, 'timings': {'bert_input_prep': 0.003237485885620117, 'sequence_prep': 0.009516000747680664, 'model_processing': 3.2185821533203125}}


# Inference

In [ ]:
# print(run_inference("play the beatles on apple music", '/content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/bert-large-uncased', model))


In [ ]:
# print(run_inference("start a one hour timer and name it shabadoo", '/content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/bert-large-uncased', model))
